# ハイパーパラメータ自動調整いろいろ
## Kerasと遺伝アルゴリズム

https://qiita.com/cvusk/items/1f3b178f34c39beb29ff#_reference-3c5df6e5237c0c705690

https://github.com/shibuiwilliam/keras_opt/blob/master/ga_nn.ipynb


In [3]:
import numpy as np
import pandas as pd
import random

from keras.layers import Activation, Dropout, BatchNormalization, Dense
from keras.models import Sequential
from keras.datasets import mnist
from keras.metrics import categorical_crossentropy
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [4]:
# MNIST class
class MNIST():
    def __init__(self,
                 l1_out=512, 
                 l2_out=512, 
                 l1_drop=0.2, 
                 l2_drop=0.2, 
                 bn1=0,
                 bn2=0,
                 batch_size=100, 
                 epochs=10, 
                 validation_split=0.1):
        self.l1_out = l1_out
        self.l2_out = l2_out
        self.l1_drop = l1_drop
        self.l2_drop = l2_drop
        self.bn1 = bn1
        self.bn2 = bn2
        self.batch_size = batch_size
        self.epochs = epochs
        self.validation_split = validation_split
        self.__x_train, self.__x_test, self.__y_train, self.__y_test = self.mnist_data()
        self.__model = self.mnist_model()
        params = """
        l1_out:\t{0}
        l2_out:\t{1}
        l1_drop:\t{2}
        l2_drop:\t{3}
        bn1:\t{4}
        bn2:\t{5}
        batch_size:\t{6}
        epochs:\t{7}
        validation_split:\t{8}
        """.format(self.l1_out, self.l2_out,
                   self.l1_drop, self.l2_drop,
                   self.bn1, self.bn2,
                   self.batch_size, self.epochs,
                   self.validation_split)
        print(params)
        
    # load mnist data from keras dataset
    def mnist_data(self):
        (X_train, y_train), (X_test, y_test) = mnist.load_data()
        X_train = X_train.reshape(60000, 784)
        X_test = X_test.reshape(10000, 784)

        X_train = X_train.astype('float32')
        X_test = X_test.astype('float32')
        X_train /= 255
        X_test /= 255

        Y_train = np_utils.to_categorical(y_train, 10)
        Y_test = np_utils.to_categorical(y_test, 10)
        return X_train, X_test, Y_train, Y_test
    
    # mnist model
    def mnist_model(self):
        model = Sequential()
        model.add(Dense(self.l1_out, input_shape=(784,)))
        if self.bn1 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l1_drop))
        model.add(Dense(self.l2_out))
        if self.bn2 == 0:
            model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Dropout(self.l2_drop))
        model.add(Dense(10))
        model.add(Activation('softmax'))
        model.compile(loss='categorical_crossentropy',
                      optimizer=Adam(),
                      metrics=['accuracy'])

        return model
    
    # fit mnist model
    def mnist_fit(self):
        early_stopping = EarlyStopping(patience=0, verbose=1)
        
        self.__model.fit(self.__x_train, self.__y_train,
                       batch_size=self.batch_size,
                       epochs=self.epochs,
                       verbose=0,
                       validation_split=self.validation_split,
                       callbacks=[early_stopping])
    
    # evaluate mnist model
    def mnist_evaluate(self):
        self.mnist_fit()
        
        evaluation = self.__model.evaluate(self.__x_test, self.__y_test, batch_size=self.batch_size, verbose=0)
        return evaluation

In [5]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()
print("X_train", X_train.shape, X_train[0, :6, :6])
print("y_train", y_train.shape, y_train[:6])

X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
print(X_train.shape)
print(X_train[0, :6])

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
print(X_train.shape)
print(X_train[0, :6])

X_train /= 255
X_test /= 255
print(X_train.shape)
print(X_train[0, :6])


11493376/11490434 [==============================] - 4s 0us/step
X_train (60000, 28, 28) [[0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
y_train (60000,) [5 0 4 1 9 2]
(60000, 784)
[0 0 0 0 0 0]
(60000, 784)
[0. 0. 0. 0. 0. 0.]
(60000, 784)
[0. 0. 0. 0. 0. 0.]


In [6]:
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

print("y_train", type(y_train), y_train.shape, y_train[0])

print("Y_train", type(Y_train), Y_train.shape, Y_train[0])


y_train <class 'numpy.ndarray'> (60000,) 5
Y_train <class 'numpy.ndarray'> (60000, 10) [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


In [7]:
def run_mnist(bounds):
    _mnist = MNIST(l1_out=bounds[0], 
                   l2_out=bounds[1], 
                   l1_drop=bounds[2], 
                   l2_drop=bounds[3], 
                   bn1=bounds[4],
                   bn2=bounds[5],
                   batch_size=bounds[6],
                   epochs=bounds[7], 
                   validation_split=bounds[8])
    mnist_evaluation = _mnist.mnist_evaluate()
    print(mnist_evaluation)
    return mnist_evaluation[0],

In [8]:
from deap import base, creator, tools, algorithms

In [16]:
creator.create('FitnessMax', base.Fitness, weights = (-1.0, ))
creator.create('Individual', list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register('L1_out', random.choice, (64, 128, 256, 512, 1024))
toolbox.register('L2_out', random.choice, (64, 128, 256, 512, 1024))
toolbox.register("L1_drop", random.uniform, 0.0, 0.3)
toolbox.register("L2_drop", random.uniform, 0.0, 0.3)
toolbox.register("bn1", random.randint, 0, 1)
toolbox.register("bn2", random.randint, 0, 1)
toolbox.register("batch_size", random.choice, (10, 100, 500))
toolbox.register("epochs", random.choice, (5, 10, 20))
toolbox.register("validation_split", random.uniform, 0.0, 0.3)

toolbox.register('individual', tools.initCycle, creator.Individual,
                 (toolbox.L1_out, toolbox.L2_out,
                 toolbox.L1_drop, toolbox.L2_drop,
                 toolbox.bn1, toolbox.bn2,
                 toolbox.batch_size, toolbox.epochs, toolbox.validation_split),
                 n=1)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpd=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", run_mnist)


/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/yuki/anaconda3/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
